<a href="https://colab.research.google.com/github/ghirailghiro/GPU_Computing_Project/blob/8-fine-tuning-and-experiments/Gradient_First_Step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Cuda plugin

!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!nvidia-smi

# plugin for cpp sintax highlighting

!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter
Sat Mar  9 17:24:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |

In [3]:
!sudo apt update
!sudo apt install -y build-essential
!sudo apt install -y libopencv-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,561 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,994 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/res

In [4]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!ls '/content/drive/My Drive/GPU Computing/human detection dataset'

0  1


In [78]:
%%writefile gradient_computation.cu
#include <opencv2/opencv.hpp>
#include <fstream>
#include <sstream>
#include <iostream>
#include <vector>
#include <filesystem>

namespace fs = std::filesystem;

void saveDescriptorAsCSVHeader(const std::vector<int>& descriptor, const std::string& filename, const std::string& label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << "x" <<descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}


void saveDescriptorAsCSV(const std::vector<float>& descriptor, const std::string& filename, int label) {
    std::ofstream file(filename, std::ios::app);
    if (!file.is_open()) {
        std::cerr << "Error: Unable to open file " << filename << " for writing." << std::endl;
        return;
    }

    // Write the descriptor to the CSV file
    for (size_t i = 0; i < descriptor.size(); ++i) {
        file << descriptor[i];
        if (i < descriptor.size() - 1) {
            file << ",";
        }
    }
    file << "," << label << "\n";
    file.close();
}

__global__ void computeGradients(unsigned char* image, float *d_magnitude, float *d_orientation, float *d_histograms, int width, int height, int cellSize) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int idy = blockIdx.y * blockDim.y + threadIdx.y;
    int indexCurrent = idy * width + idx;

    if (idx >= width || idy >= height) return; // Boundary check

    float G_x = 0;
    if (idx > 0 && idx < width - 1) {
        G_x = (float)image[idy * width + (idx + 1)] - (float)image[idy * width + (idx - 1)];
    }

    float G_y = 0;
    if (idy > 0 && idy < height - 1) {
        G_y = (float)image[(idy + 1) * width + idx] - (float)image[(idy - 1) * width + idx];
    }

    d_magnitude[indexCurrent] = sqrtf(G_x * G_x + G_y * G_y);
    d_orientation[indexCurrent] = atan2f(G_y, G_x);

    // Compute histogram bin for the current gradient
    int cellX = idx / cellSize;
    int cellY = idy / cellSize;
    int histIndex = cellY * (width / cellSize) + cellX;
    int numBins = 9; // Assuming 9 orientation bins
    float binWidth = M_PI / numBins;
    int bin = floor((d_orientation[indexCurrent] + M_PI) / binWidth);
    if (bin == numBins) bin = 0; // Wrap around
    atomicAdd(&d_histograms[histIndex * numBins + bin], d_magnitude[indexCurrent]);

    /*if((idx == 2 && idy == 2) || (idx==300 & idy == 300)){
      printf("Value for idx %d e idy %d - bin value: %d - orientation value %f \n", idx, idy, bin,d_orientation[indexCurrent] + M_PI);
    }*/
}

std::vector<float> computeDescriptors(const std::string& image_path){
        // Example: Load an image using OpenCV
    cv::Mat imageBeforeResize = cv::imread(image_path, cv::IMREAD_GRAYSCALE);
    cv::Mat image;
    cv::resize(imageBeforeResize, image, cv::Size(448, 448)); // Resize to standard size
    if(image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
        return std::vector<float>();
    }

    unsigned char* d_image;
    size_t imageSize = image.total() * image.elemSize();
    cudaMalloc(&d_image, imageSize);
    cudaMemcpy(d_image, image.data, imageSize, cudaMemcpyHostToDevice);
    size_t sizeInBytes = image.total() * sizeof(float);
    float* d_magnitude;
    cudaError_t status = cudaMalloc((void **)&d_magnitude, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_magnitude, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    sizeInBytes = image.total() * sizeof(float);
    float* d_orientation;
    status = cudaMalloc((void **)&d_orientation, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error (e.g., printing an error message and exiting)
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_orientation, 0, sizeInBytes);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }

    // Assuming image dimensions are reasonable for a grid of threads
    dim3 blockSize(16, 16);
    dim3 gridSize((image.cols + blockSize.x - 1) / blockSize.x,
                  (image.rows + blockSize.y - 1) / blockSize.y);

    // Allocate memory for histograms
    int cellSize = 128;
    int numCellsX = image.cols / cellSize;
    int numCellsY = image.rows / cellSize;
    size_t histSize = numCellsX * numCellsY * 9 * sizeof(float);
    float* d_histograms;
    status = cudaMalloc((void **)&d_histograms, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMalloc failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }
    status = cudaMemset(d_histograms, 0, histSize);
    if (status != cudaSuccess) {
        // Handle error
        fprintf(stderr, "cudaMemset failed: %s\n", cudaGetErrorString(status));
        exit(EXIT_FAILURE);
    }

    // Launch the kernel
    computeGradients<<<gridSize, blockSize>>>(d_image, d_magnitude, d_orientation, d_histograms, image.cols, image.rows, cellSize);

    cudaDeviceSynchronize();

    // Transfer histogram data from device to host
    float* h_histograms = new float[numCellsX * numCellsY * 9];
    cudaMemcpy(h_histograms, d_histograms, histSize, cudaMemcpyDeviceToHost);

    /*for (int i = 0; i < numCellsX * numCellsY; ++i) {
        for (int j = 0; j < 9; ++j) {
            std::cout << "Cell " << i << ", Bin " << j << ": " << h_histograms[i * 9 + j] << std::endl;
        }
    }*/
    // Cleanup

    // Normalization
    for (int i = 0; i < numCellsX * numCellsY; ++i) {
        float sum = 0.0f;
        for (int j = 0; j < 9; ++j) {
            sum += h_histograms[i * 9 + j] * h_histograms[i * 9 + j];
        }
        sum = sqrtf(sum);
        for (int j = 0; j < 9; ++j) {
            h_histograms[i * 9 + j] /= (sum + 1e-6); // Small constant added to avoid division by zero
        }
    }

    // Block Formation and Descriptor Computation
    std::vector<float> descriptor;
    for (int i = 0; i < numCellsY - 1; ++i) {
        for (int j = 0; j < numCellsX - 1; ++j) {
            // Concatenate histograms of four cells into a block
            for (int y = i; y < i + 2; ++y) {
                for (int x = j; x < j + 2; ++x) {
                    for (int k = 0; k < 9; ++k) {
                        descriptor.push_back(h_histograms[(y * numCellsX + x) * 9 + k]);
                    }
                }
            }
        }
    }
    cudaFree(d_image);
    cudaFree(d_magnitude);
    cudaFree(d_orientation);
    delete[] h_histograms;
    cudaFree(d_histograms);

    return descriptor;
}

int main() {

    std::string folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1"; // Change this to your folder path
    std::vector<int> header;
    //1296, 6084
    for (int i=1; i <= 144; ++i){
      header.push_back(i);
    }
    saveDescriptorAsCSVHeader(header, "descriptor.csv", "label");
    header.clear();
    //People present class
    //int count = 0;
    for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;
        /*if(count == 4){
           break;
        }
        count++;*/

        std::vector<float> descriptor = computeDescriptors(file_path);
        if (descriptor.empty()) {
            std::cout << "Vector is empty" << std::endl;
        } else {
            int label = 1;
            std::vector<float> descriptor1;
            int length = static_cast<int>(descriptor.size());
            std::cout << "Dimension of descriptor: " << length << std::endl;
            descriptor1.push_back(length);
            saveDescriptorAsCSV(descriptor, "descriptor.csv", label);
            descriptor.clear(); // Clear the vector
            descriptor1.clear();
        }
    }
    //return 0;

      //Not people present class
      folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/0";
      for (const auto& entry : fs::directory_iterator(folder_path)) {
        std::string file_path = entry.path().string();
        std::cout << "Processing image: " << file_path << std::endl;


        std::vector<float> descriptor = computeDescriptors(file_path);
        if (descriptor.empty()) {
            std::cout << "Vector is empty" << std::endl;
        } else {
            int label = 0;
            std::vector<float> descriptor1;
            int length = static_cast<int>(descriptor.size());
            std::cout << "Dimension of descriptor: " << length << std::endl;
            descriptor1.push_back(length);
            saveDescriptorAsCSV(descriptor, "descriptor.csv", label);
            descriptor.clear(); // Clear the vector
            descriptor1.clear();
        }

    }


    return 0;
}


Overwriting gradient_computation.cu


In [79]:
!nvcc gradient_computation.cu -o gradient_computation `pkg-config --cflags --libs opencv4` -run


/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [42]:
!./gradient_computation

Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/110.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/111.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/100.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/108.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/105.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/118.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/1.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Computing/human detection dataset/1/115.png
Dimension of descriptor: 144
Processing image: /content/drive/My Drive/GPU Comp

In [48]:
import os

def count_files(folder_path):
  """Counts the number of files in a directory (folder).

  Args:
      folder_path: The path to the directory.

  Returns:
      The number of files in the directory.
  """
  num_files = 0
  for entry in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, entry)):
      num_files += 1
  return num_files

# Example usage
folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/1"
number_of_files = count_files(folder_path)
print(f"There are {number_of_files} files in the folder.")

folder_path = "/content/drive/My Drive/GPU Computing/human detection dataset/0"
number_of_files1 = count_files(folder_path)
print(f"There are {number_of_files} files in the folder.")

print(f"Sum files {number_of_files + number_of_files1 }")

There are 559 files in the folder.
There are 559 files in the folder.
Sum files 921


In [80]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Assuming df is your DataFrame with features and labels
# Splitting data into features (X) and labels (y)

df = pd.read_csv("descriptor.csv")
new_value = 0

df.iloc[-1, df.columns.get_loc('label')] = new_value

X = df.drop(columns=['label'])  # Replace 'target_column' with the name of your target column
y = df['label']


In [81]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,label
0,0.603790,0.128889,0.114543,0.273055,0.549856,0.247056,0.217565,0.206709,0.278756,0.475793,...,0.435532,0.251960,0.254045,0.305447,0.378850,0.289431,0.283468,0.341170,0.406131,1
1,0.873513,0.068392,0.049734,0.078293,0.142127,0.080787,0.079607,0.095491,0.426042,0.725091,...,0.651249,0.269375,0.180607,0.186913,0.222774,0.177641,0.202935,0.303562,0.470360,1
2,0.235631,0.041689,0.061212,0.212811,0.938480,0.089825,0.049119,0.026307,0.042269,0.090080,...,0.230540,0.138172,0.200145,0.287400,0.603942,0.461299,0.359745,0.270234,0.158484,1
3,0.895189,0.153571,0.121038,0.206230,0.250040,0.062394,0.051150,0.072566,0.208747,0.757494,...,0.749571,0.228747,0.183699,0.226015,0.256271,0.167248,0.164972,0.256458,0.338167,1
4,0.738575,0.093675,0.095683,0.202665,0.479213,0.158194,0.111569,0.115040,0.339339,0.606773,...,0.555313,0.259261,0.353754,0.359959,0.469604,0.210769,0.189156,0.177758,0.193313,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,0.430024,0.094144,0.097951,0.122599,0.836963,0.122522,0.095620,0.106360,0.213582,0.316876,...,0.368007,0.163377,0.147716,0.273523,0.684443,0.422623,0.117811,0.109772,0.261230,0
917,0.609615,0.067144,0.105202,0.184390,0.176920,0.084614,0.342540,0.378184,0.529130,0.801444,...,0.854700,0.131051,0.174385,0.082154,0.222206,0.082689,0.271417,0.254607,0.143001,0
918,0.473367,0.253192,0.197594,0.444573,0.367201,0.142612,0.182925,0.291997,0.448587,0.202790,...,0.413104,0.048765,0.304419,0.467901,0.685351,0.104951,0.182321,0.034903,0.013575,0
919,0.164168,0.003464,0.018690,0.023470,0.983398,0.060630,0.036186,0.005342,0.007212,0.096823,...,0.588982,0.146562,0.131464,0.252661,0.593167,0.213251,0.168658,0.139810,0.324320,0


In [ ]:
y

0    1
1    1
2    1
3    0
Name: label, dtype: int64

In [82]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a Linear Support Vector Machine
svm = LinearSVC()
svm.fit(X_train, y_train)

# Making predictions
y_pred = svm.predict(X_test)

# Evaluating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7297297297297297
